![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [1]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : "<YOUR IBM CLOUD API KEY HERE>"
	}

# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [2]:
model_id = "ibm/granite-13b-instruct-v2"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [3]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 200,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [4]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")

## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [5]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


## Defining the inferencing input
Foundation model inferencing API accepts a natural language input that it will use
to provide the natural language response. The API is sensitive to formatting. Input
structure, presence of training steps (one-shot, two-shot learning etc.), as well
as phrasing all influence the final response and belongs to the emerging discipline of
Prompt Engineering.

Let us provide the input we got from the Prompt Lab:


,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Ground Truth
0,Westminster Bridge Road Lambeth London SE1 7UT...,2623,6/10/2017,8.7,Park Plaza Westminster Bridge London,United States of America,0,12158,I loved my stay here Did an upgrade and Big B...,27,1,10.0,"[' Leisure trip ', ' Couple ', ' Superior Doub...",54 days,51.500961,-0.116591,Positive
1,78 84 Sussex Gardens Westminster Borough Londo...,636,9/29/2016,7.7,Shaftesbury Metropolis London Hyde Park,United Kingdom,54,2867,This was our worst nightmare We have never bo...,2,1,2.5,"[' Leisure trip ', ' Family with young childre...",308 day,51.516689,-0.170615,Negative
2,Avenida Paralelo 57 59 Sants Montju c 08004 Ba...,74,8/12/2016,7.8,Tryp Barcelona Apolo Hotel,United States of America,63,883,Very dirty Mold all over bath tub Towels are ...,16,1,2.9,"[' Leisure trip ', ' Group ', ' Double or Twin...",356 day,41.374703,2.170711,Negative
3,12 Lancaster Gate Lancaster Gate Westminster B...,440,3/15/2017,8.0,Best Western Mornington Hotel Hyde Park,Norway,18,2418,Clean rooms good breakfast and friendly staff...,24,13,8.8,"[' Leisure trip ', ' Group ', ' Twin Room ', '...",141 day,51.511993,-0.178279,Positive
4,80 86 Belgrave Road Westminster Borough London...,391,10/26/2016,8.6,The Belgrave Hotel,Greece,10,2685,Great location only 5 min walk to the undergr...,13,1,9.2,"[' Leisure trip ', ' Couple ', ' Executive Dou...",281 day,51.490105,-0.138135,Positive
5,Ronda Sant Pere 9 Eixample 08010 Barcelona Spain,198,5/28/2016,9.4,Catalonia Square 4 Sup,Canada,8,1711,Very friendly and efficient staff Enjoyed the...,23,2,10.0,"[' Leisure trip ', ' Solo traveler ', ' Double...",432 day,41.388566,2.171299,Positive
6,50 Lancaster Gate Westminster Borough London W...,417,3/2/2017,6.7,Commodore Hotel,United Kingdom,27,2400,Dirty room noise of boiler all night freezing...,15,1,2.9,"[' Business trip ', ' Couple ', ' Double Room ...",154 day,51.512450,-0.179522,Negative
7,Johannesgasse 28 03 Landstra e 1030 Vienna Aus...,118,10/6/2015,8.1,InterContinental Wien,Romania,33,737,I liked the view to stadpark I liked the club...,35,3,9.2,"[' Leisure trip ', ' Couple ', ' Club Double o...",667 day,48.201986,16.378993,Positive
8,29 31 Gower Street Camden London WC1E 6HG Unit...,457,8/21/2016,6.8,Bloomsbury Palace Hotel,United Kingdom,75,2751,The hotel was dirty the rooms were stifling h...,18,1,2.9,"[' Leisure trip ', ' Couple ', ' Double Room '...",347 day,51.520795,-0.131084,Negative
9,38 Rue Du Faubourg Saint Honore 8th arr 75008 ...,118,5/9/2017,7.2,H tel De Castiglione,United Kingdom,26,1168,Everything The staff were rude and not helpfu...,0,2,2.5,"[' Leisure trip ', ' Solo traveler ', ' Superi...",86 days,48.869176,2.320581,Negative


## Execution
Let us now use the defined Model object and pair it with input and
generate the response:


In [10]:
# df_1 is your DataFrame containing the columns 'Instruction' and 'Content'
for index, row in df_1.iterrows():
    prompt_instruct = 'Analyze the sentiment expressed in the following text related to hotel review. Determine whether the sentiment is Positive or Negative. Do not include explanation.'
    prompt_input = row['Review']
    prompt_text = f"""{prompt_instruct}

    Input: {prompt_input}
    Output:"""
    
    print("Submitting generation request...")
    generated_response = model.generate_text(prompt=prompt_text, guardrails=False)
    print(generated_response)
    
    # Add a newline character for spacing
    print()

    df_1.at[index, 'Sentiment'] = generated_response

Submitting generation request...
Positive

Submitting generation request...
Negative

Submitting generation request...
Negative

Submitting generation request...
Positive

Submitting generation request...
Positive

Submitting generation request...
Positive

Submitting generation request...
Negative

Submitting generation request...
Positive

Submitting generation request...
Negative

Submitting generation request...
Negative

Submitting generation request...
Negative

Submitting generation request...
Positive

Submitting generation request...
Positive

Submitting generation request...
Negative

Submitting generation request...
Positive

Submitting generation request...
Negative

Submitting generation request...
Positive

Submitting generation request...
Negative

Submitting generation request...
Negative

Submitting generation request...
Positive



In [11]:
# Calculate accuracy
total_samples = len(df_1)
correct_predictions = (df_1['Ground Truth'] == df_1['Sentiment']).sum()
accuracy = correct_predictions / total_samples

print("Accuracy:", accuracy)

Accuracy: 1.0


In [12]:
#The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project

project = Project(None, project_id, '<YOUR PROJECT ACCESS TOKEN HERE>')
pc = project.project_context

project.save_data(data=df_1.to_csv(index=False), file_name='Hotel_review_feedback_classification_granite_instruct.csv', overwrite=True)

{'file_name': 'Hotel_review_feedback_classification_granite_instruct.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'test-donotdelete-pr-lizusgn8sbs8ci',
 'asset_id': '6b2f00bc-4275-4237-8838-d0b83f45514c'}

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  